<a href="https://colab.research.google.com/github/marcoantonio135/03MIAR-Algoritmos-de-Optimizacion/blob/main/Algoritmos%20-%20Marco%20Antonio%20Arcidiacono%20Alepuz%20-%20Trabajo%20Pr%C3%A1ctico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Marco Antonio Arcidiacono Alepuz<br>
Url: https://github.com/marcoantonio135/03MIAR-Algoritmos-de-Optimizacion<br>
Problema: Organizar los horarios de partidos de La Liga


Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia.

(*) La respuesta es obligatoria







**(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?** <br>

Respuesta:

El problema se puede ver como una variación con repetición, donde cada partido puede tener asignados 10 valores distintos, es decir, puede tomar 10 valores distintos como una cadena de 10 bits que puede tomar 2 valores distintos y sus posibles valores son $2^{10}$. Por lo tanto, las posibilidades de este problema son $10^{10}$.

**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?**<br>

Respuesta:

No hay restricciones, el objetivo es maximizar las audiencias asignando horarios a los partidos pudiendo coincidir, por lo que las posibilidades son las mismas que en la anterior pregunta: $10^{10}$.




**Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)**


Respuesta:

La estructura de datos que mejor se adapta al problema sería una tabla donde aparezcan los equipos y sus categorías, otra tabla con las categorías y las audiencias que conforman en un partido entre cada categoría, otra tabla con los horarios y sus respectivas ponderaciones, una lista con las ponderaciones de las coincidencias y finalmente una tabla con los partidos, sus horarios y las audiencias calculadas.

In [1]:
import pandas as pd
import numpy as np

categorias_equipos = pd.DataFrame(index = ['Real Madrid', 'Barcelona', 'Real Sociedad', 'Celta', 'Valencia', 'Athletic', 'Villarreal', 'Alaves', 'Levante', 'Espanyol', 'Sevilla', 'Betis', 'Atletico', 'Getafe', 'Mallorca', 'Eibar', 'Leganes', 'Osasuna', 'Granada', 'Valladolid'],
                  columns=['Categoria'])
categorias_equipos.index.name='Equipos'
categorias_equipos.iloc[:3] = 'A'
categorias_equipos.iloc[3:14] = 'B'
categorias_equipos.iloc[14:] = 'C'
categorias_equipos

,Categoria
Equipos,
Real Madrid,A
Barcelona,A
Real Sociedad,A
Celta,B
Valencia,B
Athletic,B
Villarreal,B
Alaves,B
Levante,B


In [2]:
categorias_audiencias = pd.DataFrame(data =[[2.0, 1.3, 1.0], [1.3, 0.9, 0.75], [1.0, 0.75, 0.47]],index = ['A', 'B', 'C'], columns=['A', 'B', 'C'])
categorias_audiencias

,A,B,C
A,2.0,1.30,1.00
B,1.3,0.90,0.75
C,1.0,0.75,0.47


In [3]:
ponderaciones_horarios = pd.DataFrame(data =[0.4, 0.55, 0.7, 0.8, 1.0, 0.45, 0.75, 0.85, 1.0, 0.4],index = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20'], columns=['Ponderacion'])
ponderaciones_horarios

,Ponderacion
V20,0.40
S12,0.55
S16,0.70
S18,0.80
S20,1.00
D12,0.45
D16,0.75
D18,0.85
D20,1.00
L20,0.40


In [4]:
coincidencias =[1.0, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2, 0.0, 0.0]

In [5]:
audiencias_partidos = pd.DataFrame(columns = ['Local', 'Visitante', 'CatLocal', 'CatVisitante', 'Horario', 'Coincidencias', 'Audiencia'])
audiencias_partidos['Local'] = ['Celta', 'Valencia', 'Mallorca', 'Athletic', 'Leganes', 'Villarreal', 'Alaves', 'Espanyol', 'Betis', 'Atletico']
audiencias_partidos['Visitante'] = ['Real Madrid', 'Real Sociedad', 'Eibar', 'Barcelona', 'Osasuna', 'Granada', 'Levante', 'Sevilla', 'Valladolid', 'Getafe']
audiencias_partidos['CatLocal'] = categorias_equipos.loc[audiencias_partidos['Local'], 'Categoria'].values
audiencias_partidos['CatVisitante'] = categorias_equipos.loc[audiencias_partidos['Visitante'], 'Categoria'].values
audiencias_partidos

,Local,Visitante,CatLocal,CatVisitante,Horario,Coincidencias,Audiencia
0,Celta,Real Madrid,B,A,NaN,NaN,NaN
1,Valencia,Real Sociedad,B,A,NaN,NaN,NaN
2,Mallorca,Eibar,C,C,NaN,NaN,NaN
3,Athletic,Barcelona,B,A,NaN,NaN,NaN
4,Leganes,Osasuna,C,C,NaN,NaN,NaN
5,Villarreal,Granada,B,C,NaN,NaN,NaN
6,Alaves,Levante,B,B,NaN,NaN,NaN
7,Espanyol,Sevilla,B,B,NaN,NaN,NaN
8,Betis,Valladolid,B,C,NaN,NaN,NaN
9,Atletico,Getafe,B,B,NaN,NaN,NaN


**Según el modelo para el espacio de soluciones**<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta:

Es un problema de maximización cuya función objetivo es la suma de la columna de audiencias, es decir, la audiencia total de la jornada:

$$\max\sum_{i=1}^{n}{A_i·h·c}$$

Donde $n$ es el número de partidos, $A_i$ es la audiencia del partido, $h$ es la ponderación por horario y $c$ es la ponderación por número de partidos que coinciden con ese.

**Diseña un algoritmo para resolver el problema por fuerza bruta**

Respuesta:

Un algoritmo por fuerza bruta consiste en un algoritmo que explore todas las posibles soluciones y elija la mejor. Como hemos visto anteriormente, tenemos que explorar $10^{10}$ soluciones, lo cual nos fuerza a hacer 10 bucles for anidados para explorar todas las soluciones. Después, hay que anidar otros 2 bucles for para buscar coincidencias de horarios y finalmente obtener las audiencias de cada partido y total de la jornada.

In [ ]:
#No ejecutar, tarda más de 7 horas y media
solucion = audiencias_partidos.copy()
audiencia_max = 0.0
for a in ponderaciones_horarios.index:
  audiencias_partidos.loc[0, 'Horario'] = a
  for b in ponderaciones_horarios.index:
    audiencias_partidos.loc[1, 'Horario'] = b
    for c in ponderaciones_horarios.index:
      audiencias_partidos.loc[2, 'Horario'] = c
      for d in ponderaciones_horarios.index:
        audiencias_partidos.loc[3, 'Horario'] = d
        for e in ponderaciones_horarios.index:
          audiencias_partidos.loc[4, 'Horario'] = e
          for f in ponderaciones_horarios.index:
            audiencias_partidos.loc[5, 'Horario'] = f
            for g in ponderaciones_horarios.index:
              audiencias_partidos.loc[6, 'Horario'] = g
              for h in ponderaciones_horarios.index:
                 audiencias_partidos.loc[7, 'Horario'] = h
                 for i in ponderaciones_horarios.index:
                    audiencias_partidos.loc[8, 'Horario'] = i
                    for j in ponderaciones_horarios.index:
                      audiencias_partidos.loc[9, 'Horario'] = j
                      for m in range(len(audiencias_partidos.index)):
                        audiencias_partidos.loc[m, 'Coincidencias'] = 0

                      for k in range(len(audiencias_partidos.index)):
                        for l in range(k+1, len(audiencias_partidos.index)):
                          if audiencias_partidos.loc[k, 'Horario'] == audiencias_partidos.loc[l, 'Horario']:
                            audiencias_partidos.loc[k, 'Coincidencias'] = 1 + audiencias_partidos.loc[k, 'Coincidencias']
                            audiencias_partidos.loc[l, 'Coincidencias'] = 1 + audiencias_partidos.loc[l, 'Coincidencias']
                      for m in range(len(audiencias_partidos.index)):
                        audiencias_partidos.loc[m, 'Audiencia'] = categorias_audiencias.loc[audiencias_partidos.loc[m, 'CatLocal'], audiencias_partidos.loc[m, 'CatVisitante']] * ponderaciones_horarios.loc[audiencias_partidos.loc[m, 'Horario'], 'Ponderacion'] * coincidencias[audiencias_partidos.loc[m, 'Coincidencias']]

                      if (audiencia_max < audiencias_partidos['Audiencia'].sum()):
                        solucion = audiencias_partidos.copy()
                        audiencia_max = audiencias_partidos['Audiencia'].sum()


print(audiencia_max)
solucion

**Calcula la complejidad del algoritmo por fuerza bruta**

Respuesta:

Como se puede observar, hay tantos bucles for como partidos que recorren los horarios. En el bucle for que está más adentro, hay otros 2 bucles for anidados recorriendo los partidos.

Por lo tanto, la complejidad del algoritmo sería exponencial $O(n^{m})$ siendo n el número de horarios y m el número de partidos.

**(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

Respuesta:

Para este algoritmo, primero se va a resolver el subproblema de buscar las mejores combinaciones de ponderaciones, es decir, resolver el problema como si las audiencias de los partidos fueran todas iguales.

Para ello, vamos a partir de la solución de poner todos los partidos repartidos en las mejores ponderaciones, es decir, 5 partidos el sábado a las 20 y 5 partidos el domingo a las 20. Después, se van a ir viendo soluciones vecinas (mover un partido de horario). Por último, se va a parar en tantas iteraciones como partidos haya (10 partidos, 10 iteraciones)

In [6]:
#Celda para resetear la tabla de partidos
audiencias_partidos = pd.DataFrame(columns = ['Local', 'Visitante', 'CatLocal', 'CatVisitante', 'Horario', 'Coincidencias', 'Audiencia'])
audiencias_partidos['Local'] = ['Celta', 'Valencia', 'Mallorca', 'Athletic', 'Leganes', 'Villarreal', 'Alaves', 'Espanyol', 'Betis', 'Atletico']
audiencias_partidos['Visitante'] = ['Real Madrid', 'Real Sociedad', 'Eibar', 'Barcelona', 'Osasuna', 'Granada', 'Levante', 'Sevilla', 'Valladolid', 'Getafe']
audiencias_partidos['CatLocal'] = categorias_equipos.loc[audiencias_partidos['Local'], 'Categoria'].values
audiencias_partidos['CatVisitante'] = categorias_equipos.loc[audiencias_partidos['Visitante'], 'Categoria'].values
audiencias_partidos

,Local,Visitante,CatLocal,CatVisitante,Horario,Coincidencias,Audiencia
0,Celta,Real Madrid,B,A,NaN,NaN,NaN
1,Valencia,Real Sociedad,B,A,NaN,NaN,NaN
2,Mallorca,Eibar,C,C,NaN,NaN,NaN
3,Athletic,Barcelona,B,A,NaN,NaN,NaN
4,Leganes,Osasuna,C,C,NaN,NaN,NaN
5,Villarreal,Granada,B,C,NaN,NaN,NaN
6,Alaves,Levante,B,B,NaN,NaN,NaN
7,Espanyol,Sevilla,B,B,NaN,NaN,NaN
8,Betis,Valladolid,B,C,NaN,NaN,NaN
9,Atletico,Getafe,B,B,NaN,NaN,NaN


In [7]:
partidos_horario = pd.DataFrame(data = [0, 0, 0, 0, 5, 0, 0, 0, 5, 0], index = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20'],
                                columns = ['partidos'])
partidos_horario['ponderaciones'] = [0.4, 0.55, 0.7, 0.8, 1.0, 0.45, 0.75, 0.85, 1.0, 0.4]
partidos_horario = partidos_horario.sort_values(by = ['partidos', 'ponderaciones'], ascending = False)
max_ponderaciones = 3.0
solucion = partidos_horario.copy()
for iter in range(len(audiencias_partidos.index)):
  iter_max = 0
  if (partidos_horario['partidos'].idxmax() == partidos_horario['partidos'].idxmin()):
    partidos_horario.loc[partidos_horario.index[0], 'partidos'] = partidos_horario.loc[partidos_horario.index[0], 'partidos'] - 1
    partidos_horario.loc[partidos_horario.index[1], 'partidos'] = partidos_horario.loc[partidos_horario.index[1], 'partidos'] + 1
  else:
    partidos_horario.loc[partidos_horario[['partidos', 'ponderaciones']].idxmax(), 'partidos'] = partidos_horario.loc[partidos_horario[['partidos', 'ponderaciones']].idxmax(), 'partidos'] - 1
    partidos_horario.loc[partidos_horario['partidos'].idxmin(), 'partidos'] = partidos_horario.loc[partidos_horario['partidos'].idxmin(), 'partidos'] + 1

  partidos_horario = partidos_horario.sort_values(by = ['partidos', 'ponderaciones'], ascending = False)

  total_ponderaciones = 0.0

  for i in partidos_horario.index:
    if (partidos_horario.loc[i, 'partidos'] > 0):
      total_ponderaciones = total_ponderaciones + (coincidencias[partidos_horario.loc[i, 'partidos'] - 1] * ponderaciones_horarios.loc[i, 'Ponderacion'])* partidos_horario.loc[i, 'partidos']
  if max_ponderaciones < total_ponderaciones:
    solucion = partidos_horario.copy()
    max_ponderaciones = total_ponderaciones
print(max_ponderaciones)
print(solucion)

7.1000000000000005
     partidos  ponderaciones
D20         2           1.00
S20         2           1.00
D18         1           0.85
S18         1           0.80
D16         1           0.75
S16         1           0.70
S12         1           0.55
D12         1           0.45
V20         0           0.40
L20         0           0.40


Una vez se ha resuelto este subproblema, ahora hay que colocar los partidos en los horarios.

Para ello, se colocarán los partidos de mayor audiencia con los de mejor ponderación (ponderación de horario · ponderación de coincidencia) con el objetivo de maximizar las audiencias

In [8]:
while(solucion['partidos'].sum()>0):
  max_pond=0
  max_part=0
  max_hor=''
  for i in solucion.index:
    if (max_pond < coincidencias[solucion.loc[i, 'partidos'] - 1] * solucion.loc[i, 'ponderaciones']):
      max_pond = coincidencias[solucion.loc[i, 'partidos'] - 1] * solucion.loc[i, 'ponderaciones']
      max_part = solucion.loc[i, 'partidos']
      max_hor = i

  mejores_partidos = [-1 for i in range(max_part)]
  max_aud = [0.0 for i in range(max_part)]
  for i in audiencias_partidos.index:
    if (pd.isnull(audiencias_partidos.loc[i, 'Horario'])):
      j=0
      cambiado = False
      while (j < len(max_aud) and cambiado == False):
        if (max_aud[j] < categorias_audiencias.loc[audiencias_partidos.loc[i, 'CatLocal'], audiencias_partidos.loc[i, 'CatVisitante']]):
          max_aud[j] = categorias_audiencias.loc[audiencias_partidos.loc[i, 'CatLocal'], audiencias_partidos.loc[i, 'CatVisitante']]
          mejores_partidos[j] = i
          cambiado = True
        j = j+1
  for i in range(len(max_aud)):
    audiencias_partidos.loc[mejores_partidos[i], 'Horario'] = max_hor
    audiencias_partidos.loc[mejores_partidos[i], 'Coincidencias'] = max_part - 1
    audiencias_partidos.loc[mejores_partidos[i], 'Audiencia'] = max_aud[i] * coincidencias[max_part - 1] * ponderaciones_horarios.loc[max_hor, 'Ponderacion']

  solucion.loc[max_hor, 'partidos'] = 0

audiencias_partidos

,Local,Visitante,CatLocal,CatVisitante,Horario,Coincidencias,Audiencia
0,Celta,Real Madrid,B,A,D18,0,1.105
1,Valencia,Real Sociedad,B,A,S18,0,1.04
2,Mallorca,Eibar,C,C,S12,0,0.2585
3,Athletic,Barcelona,B,A,D20,1,0.975
4,Leganes,Osasuna,C,C,D12,0,0.2115
5,Villarreal,Granada,B,C,D16,0,0.5625
6,Alaves,Levante,B,B,D20,1,0.675
7,Espanyol,Sevilla,B,B,S20,1,0.675
8,Betis,Valladolid,B,C,S16,0,0.525
9,Atletico,Getafe,B,B,S20,1,0.675


La suma total de las audiencias sería:

In [9]:
audiencias_partidos['Audiencia'].sum()

6.7025

Por lo tanto el algoritmo quedaría de la siguiente manera, en 2 bloques:

Buscar la combinación con la suma de ponderaciones más alta:<br>
  
<blockquote>solucion = sol_inicial<br>
max_ponderaciones = ponderaciones de sol_inicial<br>
nueva_sol = solucion<br>
for i from 0 to 10 do:
  <blockquote>
  nueva_sol horario con más partidos - 1 partido <br>
  nueva_sol horario con menos partidos y mejor ponderación + 1 partido<br>
  nueva_sol ponderaciones = 0<br>
  for horario in nueva_sol horarios:
    <blockquote>nueva_sol ponderaciones = nueva_sol ponderaciones + ponderacion del horario * ponderacion de coincidencias de partidos</blockquote>
  if max_ponderaciones < nueva_sol ponderaciones:
    <blockquote> max_ponderaciones = nueva_sol ponderaciones<br>
    solucion = nueva_sol</blockquote>
  </blockquote>
</blockquote>

Una vez tenemos la solucion de los horarios, las asignamos a los partidos:

<blockquote>partidos sin horario = 10<br>
while (partidos sin horario > 0):<br>
  <blockquote>for horario in solucion:
    <blockquote>mejor_ponderacion = horario con mejor combinación de ponderacion de horario y coincidencias<br>
    n_partidos = partidos de ese horario</blockquote>

  for partido in jornada:
    <blockquote>mejores_partidos = los n_partidos con mejor audiencia sin horarios asignados</blockquote>
  asignar horario, coincidencias y calcular audiencias para los n_partidos elegidos</blockquote>

Sumar audiencias ponderadas para tener la audiencia total</blockquote>

De esta manera, en el primer bloque se ha usado la búsqueda local y en el segundo se ha resuelto el problema de manera voraz asignando las mejores ponderaciones a las mejores audiencias sin ponderar.
  

Este algoritmo mejora el algoritmo por fuerza bruta porque no explora todas las soluciones y parte de una solución que se sabe de manera trivial que es mejor que otras (poner todos los partidos en los mejores horarios es mejor que poner todos los partidos en horarios peores). A partir de aquí, se exploran soluciones vecinas que pueden mejorar esta solución inicial hasta llegar a una buena solución aunque puede no ser la óptima.









**(*)Calcula la complejidad del algoritmo**

Respuesta:

Como se ha visto en el apartado anterior, en el primer bloque la complejidad sería $n^2$ (n por el primer bucle y n por el bucle anidado), es decir, $O(n^2)$ o cuadrática siendo n el número de partidos y, en el segundo bloque sería $O(n^2)$ o cuadrática por tener 2 bucles anidados que hacen iteraciones según el número de partidos.

Por lo tanto, la complejidad del algoritmo sería $O(n^2)$.

**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**

Respuesta

In [17]:
import random

#Reseteamos la tabla de partidos
audiencias_partidos = pd.DataFrame(columns = ['Local', 'Visitante', 'CatLocal', 'CatVisitante', 'Horario', 'Coincidencias', 'Audiencia'])
lista_equipos = ['Celta', 'Valencia', 'Mallorca', 'Athletic', 'Leganes', 'Villarreal', 'Alaves', 'Espanyol', 'Betis', 'Atletico', 'Real Madrid', 'Real Sociedad', 'Eibar', 'Barcelona', 'Osasuna', 'Granada', 'Levante', 'Sevilla', 'Valladolid', 'Getafe']
#usamos shuffle para reordenar aleatoriamente la lista de equipos locales y visitantes para así, generar nuevos partidos
random.shuffle(lista_equipos)
audiencias_partidos['Local'] = lista_equipos[:10]
audiencias_partidos['Visitante'] = lista_equipos[10:20]
audiencias_partidos['CatLocal'] = categorias_equipos.loc[audiencias_partidos['Local'], 'Categoria'].values
audiencias_partidos['CatVisitante'] = categorias_equipos.loc[audiencias_partidos['Visitante'], 'Categoria'].values
audiencias_partidos

,Local,Visitante,CatLocal,CatVisitante,Horario,Coincidencias,Audiencia
0,Valladolid,Athletic,C,B,NaN,NaN,NaN
1,Granada,Celta,C,B,NaN,NaN,NaN
2,Leganes,Osasuna,C,C,NaN,NaN,NaN
3,Espanyol,Villarreal,B,B,NaN,NaN,NaN
4,Sevilla,Valencia,B,B,NaN,NaN,NaN
5,Alaves,Eibar,B,C,NaN,NaN,NaN
6,Barcelona,Real Madrid,A,A,NaN,NaN,NaN
7,Betis,Real Sociedad,B,A,NaN,NaN,NaN
8,Getafe,Atletico,B,B,NaN,NaN,NaN
9,Mallorca,Levante,C,B,NaN,NaN,NaN


**Aplica el algoritmo al juego de datos generado**

Respuesta

In [18]:
partidos_horario = pd.DataFrame(data = [0, 0, 0, 0, 5, 0, 0, 0, 5, 0], index = ['V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20'],
                                columns = ['partidos'])
partidos_horario['ponderaciones'] = [0.4, 0.55, 0.7, 0.8, 1.0, 0.45, 0.75, 0.85, 1.0, 0.4]
partidos_horario = partidos_horario.sort_values(by = ['partidos', 'ponderaciones'], ascending = False)
max_ponderaciones = 3.0
solucion = partidos_horario.copy()
for iter in range(len(audiencias_partidos.index)):
  iter_max = 0
  if (partidos_horario['partidos'].idxmax() == partidos_horario['partidos'].idxmin()):
    partidos_horario.loc[partidos_horario.index[0], 'partidos'] = partidos_horario.loc[partidos_horario.index[0], 'partidos'] - 1
    partidos_horario.loc[partidos_horario.index[1], 'partidos'] = partidos_horario.loc[partidos_horario.index[1], 'partidos'] + 1
  else:
    partidos_horario.loc[partidos_horario[['partidos', 'ponderaciones']].idxmax(), 'partidos'] = partidos_horario.loc[partidos_horario[['partidos', 'ponderaciones']].idxmax(), 'partidos'] - 1
    partidos_horario.loc[partidos_horario['partidos'].idxmin(), 'partidos'] = partidos_horario.loc[partidos_horario['partidos'].idxmin(), 'partidos'] + 1

  partidos_horario = partidos_horario.sort_values(by = ['partidos', 'ponderaciones'], ascending = False)

  total_ponderaciones = 0.0

  for i in partidos_horario.index:
    if (partidos_horario.loc[i, 'partidos'] > 0):
      total_ponderaciones = total_ponderaciones + (coincidencias[partidos_horario.loc[i, 'partidos'] - 1] * ponderaciones_horarios.loc[i, 'Ponderacion'])* partidos_horario.loc[i, 'partidos']
  if max_ponderaciones < total_ponderaciones:
    solucion = partidos_horario.copy()
    max_ponderaciones = total_ponderaciones
print(max_ponderaciones)
print(solucion)

7.1000000000000005
     partidos  ponderaciones
D20         2           1.00
S20         2           1.00
D18         1           0.85
S18         1           0.80
D16         1           0.75
S16         1           0.70
S12         1           0.55
D12         1           0.45
V20         0           0.40
L20         0           0.40


In [19]:
while(solucion['partidos'].sum()>0):
  max_pond=0
  max_part=0
  max_hor=''
  for i in solucion.index:
    if (max_pond < coincidencias[solucion.loc[i, 'partidos'] - 1] * solucion.loc[i, 'ponderaciones']):
      max_pond = coincidencias[solucion.loc[i, 'partidos'] - 1] * solucion.loc[i, 'ponderaciones']
      max_part = solucion.loc[i, 'partidos']
      max_hor = i

  mejores_partidos = [-1 for i in range(max_part)]
  max_aud = [0.0 for i in range(max_part)]
  for i in audiencias_partidos.index:
    if (pd.isnull(audiencias_partidos.loc[i, 'Horario'])):
      j=0
      cambiado = False
      while (j < len(max_aud) and cambiado == False):
        if (max_aud[j] < categorias_audiencias.loc[audiencias_partidos.loc[i, 'CatLocal'], audiencias_partidos.loc[i, 'CatVisitante']]):
          max_aud[j] = categorias_audiencias.loc[audiencias_partidos.loc[i, 'CatLocal'], audiencias_partidos.loc[i, 'CatVisitante']]
          mejores_partidos[j] = i
          cambiado = True
        j = j+1
  for i in range(len(max_aud)):
    audiencias_partidos.loc[mejores_partidos[i], 'Horario'] = max_hor
    audiencias_partidos.loc[mejores_partidos[i], 'Coincidencias'] = max_part - 1
    audiencias_partidos.loc[mejores_partidos[i], 'Audiencia'] = max_aud[i] * coincidencias[max_part - 1] * ponderaciones_horarios.loc[max_hor, 'Ponderacion']

  solucion.loc[max_hor, 'partidos'] = 0

audiencias_partidos

,Local,Visitante,CatLocal,CatVisitante,Horario,Coincidencias,Audiencia
0,Valladolid,Athletic,C,B,D16,0,0.5625
1,Granada,Celta,C,B,S20,1,0.5625
2,Leganes,Osasuna,C,C,D12,0,0.2115
3,Espanyol,Villarreal,B,B,D20,1,0.675
4,Sevilla,Valencia,B,B,D20,1,0.675
5,Alaves,Eibar,B,C,S16,0,0.525
6,Barcelona,Real Madrid,A,A,D18,0,1.7
7,Betis,Real Sociedad,B,A,S18,0,1.04
8,Getafe,Atletico,B,B,S20,1,0.675
9,Mallorca,Levante,C,B,S12,0,0.4125


Suma de audiencias:

In [20]:
audiencias_partidos['Audiencia'].sum()

7.038999999999999

**Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo**

Respuesta:

1.   https://www.vadenumeros.es/sociales/variaciones.htm
2.   https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html


**Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño**

Respuesta:

Otra manera de ver el problema es tratar de minimizar las pérdidas de audiencia, aunque seguiría siendo muy parecido al problema original.

También, en lugar de asignar audiencias a cada partido, podría ser más simple para la memoria de un algoritmo y más variado en cuanto a soluciones, asignar las audiencias a los equipos directamente, aunque esto puede provocar que la complejidad aumente.

En cuanto a algoritmos para resolver el problema, una alternativa buena son los algoritmos genéticos empezando con varias soluciones iniciales con todos los horarios asignados a partidos, combinando entre estas soluciones aunque haya coincidencias y en las mutaciones pudiendo tanto deshacer coincidencias como provocarlas.